# Lectura e instalación de librerías

En esta sección se realiza la configuración para tener acceso a los datos que se van a ejecutar en el proceso, para el **análisis descriptivo** se utilizan solo 2 librerías y pandas porque la totalidad de los datos hace posible manejarlo en memoria de manera eficiente.

In [ ]:
!pip install boto3
import boto3
import pandas as pd

s3 = boto3.resource('s3', aws_access_key_id = '',
                    aws_secret_access_key='',
                    aws_session_token='')

## Análisis descriptivo (parte 1)
realizamos un método que nos permite descargar los datos y ver los datos de las variables más relevantes que se van a utilizar por ejemplo en la primera fila:
nombre del archivo, cantidad de filas y cantidad de columnas, cantidad de productos y cantidad de usuarios.


In [ ]:
for my_bucket_object in s3.Bucket('big-data-mining').objects.all():
    if 'dataset' in my_bucket_object.key:
      s3.Bucket('big-data-mining').download_file(my_bucket_object.key, my_bucket_object.key.split('/')[2])
      df = pd.read_json(my_bucket_object.key.split('/')[2], lines=True)
      descriptivo = [my_bucket_object.key.split('/')[2],
                     df.shape,
        df['product_id'].drop_duplicates().shape[0],
        df['reviewer_id'].drop_duplicates().shape[0]]

      print(descriptivo)

In [ ]:
df['product_category'].drop_duplicates().shape

(30,)

## Análisis descriptivo Parte 2

En este caso se realiza un conteo de estrellas donde se observa que todos los archivos tienen exactamente la misma cantidad de estrellas y su disrtribución es exactamente la misma para todos

In [ ]:
import os

for data in os.listdir('.'):
  if 'dataset' in data:
    print(data)
    df = pd.read_json(data, lines=True)
    print(df.groupby('stars').count())
    print(df['stars'].describe())


### columnas a utilizar

Serán el producto, el usuario y la calificación (product_id, reviewer_id y stars)

In [ ]:
print(df.columns)

Index(['review_id', 'product_id', 'reviewer_id', 'stars', 'review_body',
       'review_title', 'language', 'product_category'],
      dtype='object')


# Lectura de JSON en TensorFlow




##Primero (Helper function to read json files)
Definimos una función que se encarga de leer los archivos json que obtenemos en la sección anterior

In [1]:
import glob
import os 

def read_json_files(pattern):
  for jsonfile in glob.glob(pattern):
    with open('./' + jsonfile) as tmpfilepointer:
      for line in tmpfilepointer:
        yield tmpfilepointer.readline()


##Segundo (Creación de un archivo TFRecord)

Con la función generadora podemos crear un tipo de dato `tf.train.Example`, para generar tfRecords a partir de allí. Tambien utilizamos los helpers functions de la documentación oficial para castear los tipos de datos que estamos trabajando a un valor consumible por un TFRecord


In [2]:
import json
import tensorflow as tf

# The following functions can be used to convert a value to a type compatible
# with tf.train.Example.

def _bytes_feature(value):
  """Returns a bytes_list from a string / byte."""
  if isinstance(value, type(tf.constant(0))):
    value = value.numpy() # BytesList won't unpack a string from an EagerTensor.
  return tf.train.Feature(bytes_list=tf.train.BytesList(value=[value]))

def _float_feature(value):
  """Returns a float_list from a float / double."""
  return tf.train.Feature(float_list=tf.train.FloatList(value=[value]))

def _int64_feature(value):
  """Returns an int64_list from a bool / enum / int / uint."""
  return tf.train.Feature(int64_list=tf.train.Int64List(value=[value]))

def generate_tf_record(generator, tfrecord_filename):
  tfrecord_writer = tf.io.TFRecordWriter(tfrecord_filename)

  for line in generator:
    try:
      json_object = json.loads(line)
      sample_tensorflowdata = tf.train.Example(features=tf.train.Features(feature={
          'review_id':_bytes_feature(json_object['review_id'].encode('utf-8')),
          'product_id':_bytes_feature(json_object['product_id'].encode('utf-8')),
          'reviewer_id':_bytes_feature(json_object['reviewer_id'].encode('utf-8')),
          'stars':_int64_feature(int(json_object['stars'])),
          'review_body':_bytes_feature(json_object['review_body'].encode('utf-8')),
          'review_title':_bytes_feature(json_object['review_title'].encode('utf-8')),
          'language':_bytes_feature(json_object['language'].encode('utf-8')),
          'product_category':_bytes_feature(json_object['product_category'].encode('utf-8')),
      }))

      tfrecord_writer.write(sample_tensorflowdata.SerializeToString())
    except Exception as e:
      print(f"Error en línea: {line}")
      print(e)
      continue
  
  tfrecord_writer.close()
  

generator = read_json_files('*_train.json')
generate_tf_record(generator, 'traindata.tfrecord')

generator = read_json_files('*_test.json')
generate_tf_record(generator, 'testdata.tfrecord')
    
  


de esta manera generamos el archivo ***data.record*** que esta serializado para que sea más fácil su consumo en la librería

## Tercero (Lectura de archivo TFRecord)

Solo para el ejercicio decidimos utilizar la lectura y la escritura de los TFRecords porque toda la información se encuentra en el mismo ambiente, estos archivos solo son necesarios para transporte de información o para generarla utilizando otra estrategia.

In [6]:
filenames = ['traindata.tfrecord']
raw_dataset = tf.data.TFRecordDataset(filenames)
raw_dataset_test = tf.data.TFRecordDataset(['testdata.tfrecord'])
raw_dataset

<TFRecordDatasetV2 shapes: (), types: tf.string>

despues podemos explorar los datos como se realizaría en tensorflow

In [7]:
for raw_record in raw_dataset.take(5):
  print(repr(raw_record))

<tf.Tensor: shape=(), dtype=string, numpy=b'\n\x9e\x02\n$\n\nproduct_id\x12\x16\n\x14\n\x12product_de_0678997\n5\n\x0breview_body\x12&\n$\n"In der Lieferung war nur Ein Akku!\n\x12\n\x08language\x12\x06\n\x04\n\x02de\n,\n\x0creview_title\x12\x1c\n\x1a\n\x18EINS statt ZWEI Akkus!!!\n&\n\x0breviewer_id\x12\x17\n\x15\n\x13reviewer_de_0783625\n\x0e\n\x05stars\x12\x05\x1a\x03\n\x01\x01\n\x1b\n\treview_id\x12\x0e\n\x0c\n\nde_0559494\n(\n\x10product_category\x12\x14\n\x12\n\x10home_improvement'>
<tf.Tensor: shape=(), dtype=string, numpy=b'\n\xc3\x03\n&\n\x0breviewer_id\x12\x17\n\x15\n\x13reviewer_de_0836478\n\x0e\n\x05stars\x12\x05\x1a\x03\n\x01\x01\n!\n\x10product_category\x12\r\n\x0b\n\tdrugstore\n\x1b\n\treview_id\x12\x0e\n\x0c\n\nde_0477884\n$\n\nproduct_id\x12\x16\n\x14\n\x12product_de_0719501\n\xe7\x01\n\x0breview_body\x12\xd7\x01\n\xd4\x01\n\xd1\x01Dachte, das w\xc3\xa4ren einfach etwas festere Binden, vielleicht gr\xc3\xb6\xc3\x9fere Always. Aber die Verpackung ist derartig riesig - w

como esto no nos da mucha información podemos realizar un diccionario de transformaciones de cada uno de los features

In [8]:
# Create a description of the features.
feature_description = {
    'review_id': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'product_id': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'reviewer_id': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'stars': tf.io.FixedLenFeature([], tf.int64, default_value=0),
    'review_body': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'review_title': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'language': tf.io.FixedLenFeature([], tf.string, default_value=''),
    'product_category': tf.io.FixedLenFeature([], tf.string, default_value=''),
}

def _parse_function(example_proto):
  # Parse the input `tf.train.Example` proto using the dictionary above.
  return tf.io.parse_single_example(example_proto, feature_description)

parsed_dataset = raw_dataset.map(_parse_function)


parsed_test_dataset = raw_dataset_test.map(_parse_function)

parsed_dataset

<MapDataset shapes: {language: (), product_category: (), product_id: (), review_body: (), review_id: (), review_title: (), reviewer_id: (), stars: ()}, types: {language: tf.string, product_category: tf.string, product_id: tf.string, review_body: tf.string, review_id: tf.string, review_title: tf.string, reviewer_id: tf.string, stars: tf.int64}>

Ahora ya podemos ver los features de la siguiente manera

In [10]:
for parsed_record in parsed_dataset.take(5):
  print(repr(parsed_record))

{'language': <tf.Tensor: shape=(), dtype=string, numpy=b'de'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'home_improvement'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'product_de_0678997'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b'In der Lieferung war nur Ein Akku!'>, 'review_id': <tf.Tensor: shape=(), dtype=string, numpy=b'de_0559494'>, 'review_title': <tf.Tensor: shape=(), dtype=string, numpy=b'EINS statt ZWEI Akkus!!!'>, 'reviewer_id': <tf.Tensor: shape=(), dtype=string, numpy=b'reviewer_de_0783625'>, 'stars': <tf.Tensor: shape=(), dtype=int64, numpy=1>}
{'language': <tf.Tensor: shape=(), dtype=string, numpy=b'de'>, 'product_category': <tf.Tensor: shape=(), dtype=string, numpy=b'drugstore'>, 'product_id': <tf.Tensor: shape=(), dtype=string, numpy=b'product_de_0719501'>, 'review_body': <tf.Tensor: shape=(), dtype=string, numpy=b'Dachte, das w\xc3\xa4ren einfach etwas festere Binden, vielleicht gr\xc3\xb6\xc3\x9fere Always. Aber d

# Creación de un Sistema de recomendación

## Preparación de datos y librerías

Utilizaremos la librería TensorFlow Recomenders que esta especializada en sistemas de recomendación

In [11]:
!pip install -q tensorflow-recommenders
!pip install -q --upgrade tensorflow-datasets
!pip install -q scann

In [12]:
import os
import pprint
import tempfile

from typing import Dict, Text

import numpy as np
import tensorflow as tf
import tensorflow_recommenders as tfrs

De nuestro análisis anterior (en Spark) obtuvimos que nuestras variables a utilizar son: _product_id_,_reviewer_id_, _stars_ 

In [13]:
parsed_dataset = parsed_dataset.map(lambda x: {
    'reviewer_id':x['reviewer_id'], 
    'product_id':x['product_id'], 
    'stars': x['stars']
  })


for x in parsed_dataset.take(1).as_numpy_iterator():
  pprint.pprint(x)


{'product_id': b'product_de_0678997',
 'reviewer_id': b'reviewer_de_0783625',
 'stars': 1}


In [14]:
train = parsed_dataset
test = parsed_test_dataset

In [15]:
test

<MapDataset shapes: {language: (), product_category: (), product_id: (), review_body: (), review_id: (), review_title: (), reviewer_id: (), stars: ()}, types: {language: tf.string, product_category: tf.string, product_id: tf.string, review_body: tf.string, review_id: tf.string, review_title: tf.string, reviewer_id: tf.string, stars: tf.int64}>

In [16]:
products = parsed_dataset.batch(100_000).map(lambda x: x["product_id"])
user_ids = parsed_dataset.batch(100_000).map(lambda x: x["reviewer_id"])

unique_products = np.unique(np.concatenate(list(products)))
unique_user_ids = np.unique(np.concatenate(list(user_ids)))

unique_products[:10]

array([b'product_de_0000011', b'product_de_0000029',
       b'product_de_0000040', b'product_de_0000053',
       b'product_de_0000060', b'product_de_0000065',
       b'product_de_0000067', b'product_de_0000070',
       b'product_de_0000122', b'product_de_0000130'], dtype=object)

In [17]:
unique_user_ids[:10]

array([b'reviewer_de_0000000', b'reviewer_de_0000013',
       b'reviewer_de_0000058', b'reviewer_de_0000088',
       b'reviewer_de_0000098', b'reviewer_de_0000105',
       b'reviewer_de_0000109', b'reviewer_de_0000122',
       b'reviewer_de_0000129', b'reviewer_de_0000144'], dtype=object)

Con esto finalizamos la preparación de datos

## Modelado

Siguiendo la documentación de tensorflow para la implementación del modelo realizamos los siguientes pasos.

In [18]:
embedding_dimension = 32 #hay que probar valores que mejor se ajusten para evitar overfitting

Basado en la documentación de tfrs, sabemos que podemos generar un modelo heredando de la clase model, entonces realizamos los siguientes ajustes. Primero creamos un modelo que hereda de la clase keras.Model de tensorflow en esta clase se transforman los datos de texto a numericos y se calcula la predicción, normalizando los datos aunque para estos no se hace tan necesario.

In [29]:
class RankingModel(tf.keras.Model):

  def __init__(self, embedding_dimension):
    super().__init__()
    embedding_dimension = embedding_dimension

    # Compute embeddings for users.
    self.user_embeddings = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_user_ids, mask_token=None),
      tf.keras.layers.Embedding(len(unique_user_ids) + 1, embedding_dimension)
    ])

    # Compute embeddings for movies.
    self.product_embeddings = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.StringLookup(
        vocabulary=unique_products, mask_token=None),
      tf.keras.layers.Embedding(len(unique_products) + 1, embedding_dimension)
    ])

    # Compute predictions.
    self.ratings = tf.keras.Sequential([
      tf.keras.layers.experimental.preprocessing.Normalization(),
      # Learn multiple dense layers.
      tf.keras.layers.Dense(256, activation="relu"),
      tf.keras.layers.Dense(64, activation="relu"),
      # Make rating predictions in the final layer.
      tf.keras.layers.Dense(1)
  ])

  def call(self, inputs):

    user_id, product_id = inputs

    user_embedding = self.user_embeddings(user_id)
    product_embedding = self.product_embeddings(product_id)

    return self.ratings(tf.concat([user_embedding, product_embedding], axis=1))

In [30]:
RankingModel(embedding_dimension)((["product_de_0000011"], ["'reviewer_de_0000098'"]))

Consider rewriting this model with the Functional API.
Consider rewriting this model with the Functional API.


<tf.Tensor: shape=(1, 1), dtype=float32, numpy=array([[-0.02045668]], dtype=float32)>

In [31]:
class AWSRatingsModel(tfrs.models.Model):

  def __init__(self, embedding_dimension):
    super().__init__()
    self.ranking_model: tf.keras.Model = RankingModel(embedding_dimension)
    self.task: tf.keras.layers.Layer = tfrs.tasks.Ranking(
      loss = tf.keras.losses.MeanSquaredError(),
      metrics=[tf.keras.metrics.RootMeanSquaredError()]
    )

  def compute_loss(self, features: Dict[Text, tf.Tensor], training=False) -> tf.Tensor:
    rating_predictions = self.ranking_model(
        (features["reviewer_id"], features["product_id"]))

    # The task computes the loss and the metrics.
    return self.task(labels=features["stars"], predictions=rating_predictions)

# Ejecución del modelo

Instanciamos la clase del modelo que acabamos de crear pasandole de parámetro los embeddings que se ajusto sobre el modelo de guía de la documentación

In [32]:
model = AWSRatingsModel(embedding_dimension)
model.compile(optimizer=tf.keras.optimizers.Adagrad(learning_rate=0.1))

In [36]:
cached_train = train.batch(200000).cache()
cached_test = test.batch(20000).cache()

In [37]:
model.fit(cached_train, epochs=6)

Epoch 1/6
2/2 [==============================] - 15s 5s/step - root_mean_squared_error: 1.4637 - loss: 2.1083 - regularization_loss: 0.0000e+00 - total_loss: 2.1083
Epoch 2/6
2/2 [==============================] - 2s 545ms/step - root_mean_squared_error: 1.4153 - loss: 2.0016 - regularization_loss: 0.0000e+00 - total_loss: 2.0016
Epoch 3/6
2/2 [==============================] - 2s 541ms/step - root_mean_squared_error: 1.4136 - loss: 1.9983 - regularization_loss: 0.0000e+00 - total_loss: 1.9983
Epoch 4/6
2/2 [==============================] - 2s 548ms/step - root_mean_squared_error: 1.4135 - loss: 1.9982 - regularization_loss: 0.0000e+00 - total_loss: 1.9982
Epoch 5/6
2/2 [==============================] - 2s 561ms/step - root_mean_squared_error: 1.4135 - loss: 1.9981 - regularization_loss: 0.0000e+00 - total_loss: 1.9981
Epoch 6/6
2/2 [==============================] - 2s 541ms/step - root_mean_squared_error: 1.4135 - loss: 1.9981 - regularization_loss: 0.0000e+00 - total_loss: 1.9981


## Evaluación del Modelo

En este paso evaluamos llos resultados de ejecución del modelo

In [38]:
model.evaluate(cached_test, return_dict=True)

1/1 [==============================] - 0s 337ms/step - root_mean_squared_error: 1.4151 - loss: 2.0026 - regularization_loss: 0.0000e+00 - total_loss: 2.0026


{'loss': 2.0025787353515625,
 'regularization_loss': 0,
 'root_mean_squared_error': 1.4151250123977661,
 'total_loss': 2.0025787353515625}